In [1]:
# python warning off
import warnings
warnings.filterwarnings('ignore')

# tensorflow warning off
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
import pandas as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [3]:
def read(what, how):
#원하는 데이터 연도별로 다 불러오가
#what은 무슨 기록인지 받는 입력   예)팀타자, 개인투수....
#how는 생성할 데이터 프레임 이름  예)df_raw, df_bat...
    year = [16,17,18,19,20]
    for i in year:
        globals()['ver1_{0}{1}'.format(how,i)] = pd.read_csv("idea1_1/ver1_{1}{0}.csv".format(what,i))
        globals()['ver2_{0}{1}'.format(how,i)] = pd.read_csv("idea2/new_{1}{0}.csv".format(what,i))

In [4]:
team = ['HH', 'SS', 'NC', 'LG', 'SK', 'LT', 'OB', 'HT', 'KT', 'WO']
teamk = ['한화','삼성','NC', 'LG', 'SK', '롯데', '두산','KIA', 'KT', '키움']
year = [16,17,18,19,20]

In [5]:
for i in team:
    read(i,i)

In [6]:
def pre_data(df):
    df= pd.concat([df, pd.get_dummies(df["VS_T_ID"],prefix='VS_T_ID')],axis=1)
    df = df.drop(columns = ['VS_T_ID'])
    df = df.reset_index(drop= True)
    return df

#pred20 = pre_data(pred20)

# for i in range(len(teamk)):
#     globals()['pred{0}'.format(team[i])]=pred20[pred20['T_ID']==teamk[i]]
#     globals()['pred{0}'.format(team[i])]['T_ID'] = team[i]

####  훈련 준비

In [7]:
ver1_col = list(ver1_HH16.columns)
ver2_col = list(ver2_HH16.columns)

In [8]:
print(ver1_col, ver2_col)

['date', '월', '일', 'VS_T_ID', 'P_ID_0', 'P_ID_1', 'P_ID_2', 'P_ID_3', 'P_ID_4'] ['first', 'second', 'third', 'fourth', 'P_ID']


In [9]:
for i in team:
    for j in year:
        globals()['{0}{1}'.format(i,j)] = pd.concat([globals()['ver2_{0}{1}'.format(i,j)][ver2_col[:-1]],
                                                               globals()['ver1_{0}{1}'.format(i,j)][ver1_col[3:]].loc[4:].reset_index(drop=True) ] ,axis = 1)

In [10]:
# df = pd.concat([ver2_HH16[ver2_col[:-1]], ver1_HH16[ver1_col[3:]].loc[4:].reset_index(drop=True) ] ,axis = 1)
# team = ['HH', 'SS', 'NC', 'LG', 'SK', 'LT', 'OB', 'HT', 'KT', 'WO']
df = pd.concat([HH16,SS16,NC16,LG16,SK16,LT16,OB16,HT16, KT16, WO16,
                  HH17,SS17,NC17,LG17,SK17,LT17,OB17,HT17, KT17, WO17,
                  HH18,SS18,NC18,LG18,SK18,LT18,OB18,HT18, KT18, WO18,
                  HH19,SS19,NC19,LG19,SK19,LT19,OB19,HT19, KT19, WO19,
                  HH20,SS20,NC20,LG20,SK20,LT20,OB20,HT20, KT20, WO20
               ], axis=0)

In [11]:
df

,first,second,third,fourth,VS_T_ID,P_ID_0,P_ID_1,P_ID_2,P_ID_3,P_ID_4
0,0,1,2,1,3,1,0,0,0,0
1,1,2,1,0,2,0,1,0,0,0
2,2,1,0,1,2,0,1,0,0,0
3,1,0,1,1,2,0,0,1,0,0
4,0,1,1,2,6,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
57,2,3,4,0,2,0,1,0,0,0
58,3,4,0,1,2,1,0,0,0,0
59,4,0,1,0,1,0,0,0,1,0
60,0,1,0,3,1,0,0,0,0,1


In [12]:
def pre_data(df):
    df= pd.concat([df, pd.get_dummies(df["VS_T_ID"],prefix='VS_T_ID')],axis=1)
    df = df.drop(columns = ['VS_T_ID'])
    df = df.reset_index(drop= True)
    return df

In [13]:
df = pre_data(df)

### 아이디어 스케치!

In [14]:
def drop(df):
    drop_index=[]
    for i in range(len(df)):
        li = list(df.loc[i,'first':'fourth'].values)
        for j in range(3):
            if li[j] == li[j+1]:
                drop_index.append(i)
                break
    for k in drop_index:
        df = df.drop(k)
    df = df.reset_index(drop= True)
    return df

In [15]:
df = drop(df)

In [16]:
label_cols= list(df.columns)[4:9]
feature_cols = list(df.columns)[:4] # +list(df.columns)[9:]
# 데이터를 묶어서 처리하는 함수
def make_dateset(data, label, window_size=10):
    import numpy as np
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size+1]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

In [17]:
TEST_SIZE = 300
train = df[:-TEST_SIZE]
test = df[-TEST_SIZE:]
#for i in team:
#    globals()['test{}'.format(i)] =  df[] 
#TEST_SIZE = 536
#train = df[:-TEST_SIZE]
#test = df[-TEST_SIZE:]

In [18]:
train_feature, train_label = make_dateset(train[feature_cols], train[label_cols],5)
test_feature, test_label = make_dateset(test[feature_cols], test[label_cols],5)

In [19]:
train_feature.shape

(5055, 6, 4)

In [20]:
VALID_SIZE = 355
x_train = train_feature[:-VALID_SIZE]
x_valid = train_feature[VALID_SIZE:]
y_train = train_label[:-VALID_SIZE]
y_valid = train_label[VALID_SIZE:]

In [21]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Dropout

In [22]:
def stacked_lstm(n_neurons=32):
    model = Sequential()
    model.add(LSTM(n_neurons, input_shape =  [train_feature.shape[1], train_feature.shape[2]], return_sequences = True, activation ='relu'))
    model.add(LSTM(n_neurons, return_sequences = False, activation ='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(5, activation ='softmax'))
   
# adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [23]:
from sklearn.model_selection import RandomizedSearchCV
param_distribs = {
    "n_neurons": np.arange(1, 100),
}

from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=10)

with tf.device('/GPU:0'):
    keras_reg = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn = stacked_lstm)
    rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)
    rnd_search_cv.fit(x_train, y_train, epochs=500, validation_data=(x_valid, y_valid),
                      callbacks=[early_stop])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] n_neurons=51 ....................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Train on 3133 samples, validate on 4700 samples
Epoch 1/500
3133/3133 [==============================] - 7s 2ms/sample - loss: 1.6082 - accuracy: 0.2247 - val_loss: 1.6001 - val_accuracy: 0.3096
Epoch 2/500
3133/3133 [==============================] - 3s 820us/sample - loss: 1.5582 - accuracy: 0.2783 - val_loss: 1.4595 - val_accuracy: 0.3606
Epoch 3/500
3133/3133 [==============================] - 2s 783us/sample - loss: 1.4490 - accuracy: 0.3645 - val_loss: 1.3693 - val_accuracy: 0.4115
Epoch 4/500
3133/3133 [==============================] - 2s 797us/sample - loss: 1.3638 - accuracy: 0.4402 - val_loss: 1.3130 - val_accuracy: 0.5196
Epoch 5/500
3133/3133 [==============================] - 3s 829us/sample - loss: 1.3315 - accuracy: 0.4657 - val_loss: 1.2796 - val_accuracy: 0.5128
Epoch 6/500
3133/3133 [==============================] - 3s 811us/sample - loss: 1.3138 - accuracy: 0.4864 - val_loss: 1.2660 - val_accuracy: 0.5043
Epoch 7/500
3133/3133 [==============================] - 3s 

3133/3133 [==============================] - 3s 864us/sample - loss: 0.8864 - accuracy: 0.6914 - val_loss: 0.9269 - val_accuracy: 0.6857
Epoch 56/500
3133/3133 [==============================] - 3s 837us/sample - loss: 0.8629 - accuracy: 0.7016 - val_loss: 0.8985 - val_accuracy: 0.6917
Epoch 57/500
3133/3133 [==============================] - 3s 858us/sample - loss: 0.8698 - accuracy: 0.6984 - val_loss: 0.9101 - val_accuracy: 0.6887
Epoch 58/500
3133/3133 [==============================] - 3s 839us/sample - loss: 0.8693 - accuracy: 0.6955 - val_loss: 0.9015 - val_accuracy: 0.6943
Epoch 59/500
3133/3133 [==============================] - 3s 856us/sample - loss: 0.8486 - accuracy: 0.7044 - val_loss: 0.8984 - val_accuracy: 0.6972
Epoch 60/500
3133/3133 [==============================] - 3s 863us/sample - loss: 0.8724 - accuracy: 0.6971 - val_loss: 0.9063 - val_accuracy: 0.6913
Epoch 61/500
3133/3133 [==============================] - 3s 827us/sample - loss: 0.8451 - accuracy: 0.7048 - val

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.9min remaining:    0.0s


...

Train on 4700 samples, validate on 4700 samples
Epoch 1/500
4700/4700 [==============================] - 10s 2ms/sample - loss: 1.6120 - accuracy: 0.2006 - val_loss: 1.6061 - val_accuracy: 0.2123
Epoch 2/500
4700/4700 [==============================] - 5s 970us/sample - loss: 1.5991 - accuracy: 0.2423 - val_loss: 1.5666 - val_accuracy: 0.2923
Epoch 3/500
4700/4700 [==============================] - 4s 800us/sample - loss: 1.4978 - accuracy: 0.3332 - val_loss: 1.3712 - val_accuracy: 0.4345
Epoch 4/500
4700/4700 [==============================] - 3s 710us/sample - loss: 1.4063 - accuracy: 0.4021 - val_loss: 1.3511 - val_accuracy: 0.4564
Epoch 5/500
4700/4700 [==============================] - 3s 716us/sample - loss: 1.3500 - accuracy: 0.4530 - val_loss: 1.2614 - val_accuracy: 0.5472
Epoch 6/500
4700/4700 [==============================] - 3s 709us/sample - loss: 1.3230 - accuracy: 0.4845 - val_loss: 1.2158 - val_accuracy: 0.6006
Epoch 7/500
4700/4700 [==============================] - 4s

4700/4700 [==============================] - 3s 688us/sample - loss: 1.0096 - accuracy: 0.6345 - val_loss: 0.9020 - val_accuracy: 0.6804
Epoch 56/500
4700/4700 [==============================] - 3s 706us/sample - loss: 0.9900 - accuracy: 0.6447 - val_loss: 0.8961 - val_accuracy: 0.6838
Epoch 57/500
4700/4700 [==============================] - 3s 696us/sample - loss: 0.9927 - accuracy: 0.6464 - val_loss: 0.9026 - val_accuracy: 0.6836
Epoch 58/500
4700/4700 [==============================] - 3s 701us/sample - loss: 0.9971 - accuracy: 0.6415 - val_loss: 0.8842 - val_accuracy: 0.6860
Epoch 59/500
4700/4700 [==============================] - 3s 715us/sample - loss: 0.9962 - accuracy: 0.6411 - val_loss: 0.9104 - val_accuracy: 0.6811
Epoch 60/500
4700/4700 [==============================] - 3s 698us/sample - loss: 0.9859 - accuracy: 0.6511 - val_loss: 0.8928 - val_accuracy: 0.6877
Epoch 61/500
4700/4700 [==============================] - 3s 720us/sample - loss: 0.9931 - accuracy: 0.6415 - val

4700/4700 [==============================] - 3s 724us/sample - loss: 0.8363 - accuracy: 0.6932 - val_loss: 0.7387 - val_accuracy: 0.7300
Epoch 110/500
4700/4700 [==============================] - 3s 714us/sample - loss: 0.8426 - accuracy: 0.6921 - val_loss: 0.7372 - val_accuracy: 0.7313
Epoch 111/500
4700/4700 [==============================] - 3s 715us/sample - loss: 0.8467 - accuracy: 0.6881 - val_loss: 0.7554 - val_accuracy: 0.7230
Epoch 112/500
4700/4700 [==============================] - 3s 742us/sample - loss: 0.8374 - accuracy: 0.6891 - val_loss: 0.7363 - val_accuracy: 0.7336
Epoch 113/500
4700/4700 [==============================] - 3s 740us/sample - loss: 0.8314 - accuracy: 0.6930 - val_loss: 0.7572 - val_accuracy: 0.7340
Epoch 114/500
4700/4700 [==============================] - 4s 750us/sample - loss: 0.8371 - accuracy: 0.6968 - val_loss: 0.7306 - val_accuracy: 0.7413
Epoch 115/500
4700/4700 [==============================] - 4s 777us/sample - loss: 0.8389 - accuracy: 0.6860

4700/4700 [==============================] - 3s 744us/sample - loss: 0.7199 - accuracy: 0.7266 - val_loss: 0.6455 - val_accuracy: 0.7777
Epoch 164/500
4700/4700 [==============================] - 3s 714us/sample - loss: 0.7019 - accuracy: 0.7381 - val_loss: 0.6276 - val_accuracy: 0.7809
Epoch 165/500
4700/4700 [==============================] - 3s 699us/sample - loss: 0.7094 - accuracy: 0.7283 - val_loss: 0.6246 - val_accuracy: 0.7857
Epoch 166/500
4700/4700 [==============================] - 3s 703us/sample - loss: 0.7177 - accuracy: 0.7272 - val_loss: 0.6324 - val_accuracy: 0.7872
Epoch 167/500
4700/4700 [==============================] - 3s 700us/sample - loss: 0.7123 - accuracy: 0.7266 - val_loss: 0.6501 - val_accuracy: 0.7751
Epoch 168/500
4700/4700 [==============================] - 3s 696us/sample - loss: 0.7150 - accuracy: 0.7351 - val_loss: 0.6363 - val_accuracy: 0.7868
Epoch 169/500
4700/4700 [==============================] - 3s 690us/sample - loss: 0.6987 - accuracy: 0.7357

4700/4700 [==============================] - 4s 763us/sample - loss: 0.6578 - accuracy: 0.7519 - val_loss: 0.5735 - val_accuracy: 0.8096
Epoch 218/500
4700/4700 [==============================] - 4s 747us/sample - loss: 0.6212 - accuracy: 0.7579 - val_loss: 0.5792 - val_accuracy: 0.8130


In [28]:
import sklearn

In [29]:
sklearn.__version__

'0.21.3'

In [30]:
tf.__version__

'2.1.0'

In [31]:
rnd_search_cv.best_params_

{'n_neurons': 29}

In [32]:
rnd_search_cv.best_score_

0.6136170238002817

In [33]:
rnd_search_cv.best_estimator_

In [35]:
rnd_search_cv.score(x_valid, y_valid)

4700/4700 [==============================] - 1s 244us/sample - loss: 0.5792 - accuracy: 0.8130


0.81297874

In [36]:
rnd_search_cv.score(test_feature, test_label)

295/295 [==============================] - 0s 264us/sample - loss: 1.2371 - accuracy: 0.7797


0.779661

In [37]:
model = rnd_search_cv.best_estimator_.model
model.evaluate(test_feature, test_label)

295/295 [==============================] - 0s 245us/sample - loss: 1.2371 - accuracy: 0.7797


[1.2371209007198527, 0.779661]

In [40]:
pred = model.predict(test_feature)
a=0
for i in range(len(pred)): 
    if np.argmax(pred[i]) == np.argmax(test_label[i]):
        a+=1
print(len(pred), a, a/len(pred))

295 230 0.7796610169491526
